In [ ]:
import datetime
from typing_extensions import TypedDict
from typing import List
import json

class PortfolioItem(TypedDict):
    ticker: str
    friendlyName: str
    amount: int
Portfolio = List[PortfolioItem]

def loadPortfolio(p: Portfolio) -> Portfolio:
    custom = 'portfolio.json'
    sample = 'portfolioSample.json'
    useSample = False
    if not os.path.isfile(custom):
        print("Using sample data.\nPlease create a portfolio.json to use custom input.")
        useSample = True
    with open(sample if useSample else custom, 'r') as infile:
        return json.load(infile)

portfolio = loadPortfolio(myPortfolio)
useDataOverYears = 5 # 1y,2y,5y,10y
endDate = datetime.date.today() # only influences parsing atm, not collection

In [ ]:
import json
import yfinance as yf
import os.path

def getDataFileName(name: str) -> str:
    return f'data/{name}.json'

def convertDataFrameToJson(df: pd.DataFrame) -> any:
    return json.loads(df.to_json())

def writeJsonToFile(name: str, payload: any) -> None:
    filename = getDataFileName(name)
    with open(filename, 'w') as outfile:
        json.dump(payload, outfile)

    print(f"{filename} saved!")
    
def getYFDataFrame(ticker: str, years: int) -> pd.DataFrame:
    dataframe = yf.download(
        ticker, 
        period=str(years)+'y',
        prepost=True,
        interval="1d" # none of the larger intervals seem to be consistent
    )
    
    return dataframe

def downloadPortfolio(portfolio: Portfolio, years: int, partial=True) -> None:
    for stock in portfolio:
        ticker = stock["ticker"]
        if not os.path.isfile(getDataFileName(ticker)) or not partial:
            print(f"Retreiving {ticker} information...")
            writeJsonToFile(
                ticker,
                convertDataFrameToJson(
                    getYFDataFrame(ticker, years)
                )
            )
        else:
            print(f"{ticker} already exists. Skipping download")

    
downloadPortfolio(portfolio, useDataOverYears)

In [ ]:
import numpy as np
import pandas as pd
import json
import datetime
import functools

def getPortfolioWeight(ticker: str) -> float:
    portfolioSum = functools.reduce(lambda acc, curr: acc + curr['amount'], portfolio, 0)
    tickerVal = 0
    for stock in portfolio:
        if stock['ticker'] == ticker:
            tickerVal = stock['amount']
            
    return tickerVal/portfolioSum
    
class PortfolioInfo(TypedDict):
    ticker: str
    name: str
    weight: float
    basePriceDate: any
    startDate: any
    basePriceMean: float
    recentMean: float
    priceDelta: float
    changePerAnnum: float
    
def parseStockDataFrame(df: pd.DataFrame, portfolioEntry: PortfolioItem) -> PortfolioInfo:
    yearBasePriceOffset = 1 # how many years should be used as the base price
    
    # initialise fundamental dates
    minDate = df.index[0]
    maxDate = endDate - datetime.timedelta(weeks=useDataOverYears*52)
    baseDate = maxDate if maxDate >= minDate else minDate
    recentDate = baseDate + datetime.timedelta(weeks=yearBasePriceOffset * 52)

    # take the mean from the first year in the dataset
    basePriceFrame = df[(df.index > baseDate.isoformat()) & (df.index < recentDate.isoformat())]
    basePriceMean = basePriceFrame["Open"].mean()

    # calculate the delta since the first year mean
    recentMean = df[df.index > recentDate.isoformat()]["Open"].mean()
    priceDelta = (recentMean - basePriceMean) / basePriceMean
    changePerAnnum = priceDelta / useDataOverYears # off by yeaBasePriceOffset?
    
    return {
        'ticker': portfolioEntry['ticker'],
        'name': portfolioEntry['friendlyName'],
        'weight': getPortfolioWeight(portfolioEntry['ticker']),
        'basePriceDate': baseDate,
        'startDate': recentDate,
        'basePriceMean': basePriceMean,
        'recentMean': recentMean,
        'priceDelta': priceDelta,
        'changePerAnnum': changePerAnnum
    }

portfolioMeans = []
for entry in portfolio:
    portfolioMeans.append(
        parseStockDataFrame(
            pd.read_json(
                getDataFileName(entry['ticker'])
            ),
            entry
        )
    )
    
    
portfolioDf = pd.DataFrame(portfolioMeans)
portfolioDf['weightedChangePerAnnum'] = portfolioDf['weight'] * portfolioDf['changePerAnnum']
portfolioDf



In [ ]:
import matplotlib.pyplot as plt

# Data to plot
labels = portfolioDf['name']
weights = portfolioDf['weight']
weightedChange = portfolioDf['weightedChangePerAnnum']
colors = ['gold', 'yellowgreen', 'lightcoral', 'lightskyblue']

# Weights
plt.pie(
    weights, 
    labels=labels, 
    colors=colors,
    autopct='%1.1f%%', 
    shadow=True, 
    startangle=140
)
plt.axis('equal')
plt.title("Portfolio Weights", pad=15)

plt.show()

# Change By Weight
plt.bar(
    labels, 
    weightedChange * 100,
    0.5
)
plt.title("Portfolio Weighted Expected Change %", pad=15)

plt.show()
print(f"Total Expected % Change is {weightedChange.sum()*100}")